In [16]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [17]:
def image_generator(directory, filenames, labels, batch_size=32, target_size=(224, 224), shuffle=True):
    num_samples = len(filenames)
    indices = np.arange(num_samples)
    if shuffle:
        np.random.shuffle(indices)

    while True:
        for start_idx in range(0, num_samples, batch_size):
            batch_indices = indices[start_idx:start_idx + batch_size]
            batch_images = []
            batch_labels = []
            for idx in batch_indices:
                filename = filenames[idx]
                label = labels[idx]
                img_path = os.path.join(directory, filename)
                img = image.load_img(img_path, target_size=target_size)
                img_array = image.img_to_array(img)
                img_array = img_array / 255.0  # Rescale to [0,1]
                batch_images.append(img_array)
                batch_labels.append(label)

            yield np.array(batch_images), np.array(batch_labels)

In [18]:
dataset_dir = "D:/yapayzekafinal/resim"  # Ana veri dizini
classes = os.listdir(dataset_dir)
num_classes = len(classes)
print(classes)

['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


In [19]:
def load_data():
    filenames = []
    labels = []
    for i, cls in enumerate(classes):
        class_dir = os.path.join(dataset_dir, cls)
        class_filenames = os.listdir(class_dir)
        class_filenames = [os.path.join(cls, filename) for filename in class_filenames]
        filenames.extend(class_filenames)
        labels.extend([i] * len(class_filenames))
    return filenames, labels

In [20]:
filenames, labels = load_data()
X_train_filenames, X_test_filenames, y_train, y_test = train_test_split(filenames, labels, test_size=0.2, random_state=42)

In [21]:
model = Sequential()

model.add(Conv2D(input_shape=((224,244,3)),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))


model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))



d:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [22]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
train_generator = image_generator(dataset_dir, X_train_filenames, y_train)
validation_generator = image_generator(dataset_dir, X_test_filenames, y_test)


In [24]:
model.fit(train_generator,
          steps_per_epoch=len(X_train_filenames) // 32,  # Eğitim veri setinizdeki toplam örnek sayısını uygun şekilde ayarlayın
          epochs=10,
          validation_data=validation_generator,
          validation_steps=len(X_test_filenames) // 32)

Epoch 1/10
  1/717 ━━━━━━━━━━━━━━━━━━━━ 6:42:24 34s/step - accuracy: 0.1562 - loss: 1.9460


KeyboardInterrupt



In [30]:
model.save("deneme1.keras")